# Inicio del proyecto y preparacion

## Web Scrapping

Para iniciar con el web scrapping, debemos primero descargar las bibliotecas bs4 y scrappy que utilizaremos para simplificar la manera de obtener la informacion.

Iniciar la terminal y ejecutar el siguiente codigo:

pip install beautifulsoup4
pip install scrapy

------------- INICIO POSIBLE PROBLEMA --------------
Si tenes algun WARNING relacionado con el PATH segui con los siguientes pasos:

Copiar el directorio que esta luego del WARNING, poner en el buscador de windows "View Advanced System Setting" -> "Environment Variables..." -> (donde dice system variables) seleccionar Path con doble click -> New y pegar el directorio anteriormente copiado.

Luego, abrir cdm (terminal) e ir hasta el directorio donde se trabaja el proyecto mediante "cd [nombre subcarpeta]" y consultando las carpetas a ingresar mediante el comando "dir".

------------- FIN POSIBLE PROBLEMA ----------------

Abrir la terminal en la carpeta del proyecto y ejecutar el comando "scrapy startproject zonaprop", luego ejecutar el comando "scrapy genspider zonaprop_ret zonaprop.com.ar".

Esto creara una estructura de carpetas en tu proyecto que debemos modificar a gusto, comencemos por mover el archivo "zonaprop_ret" a la carpeta "zonaprop/zonaprop/spiders/", y abrirlo para modificacion.

Una vez alli, investigamos la web que queremos scrappear, en este caso www.zonaprop.com.ar y damos ctrl + u para abrir la planilla de desarrollador.

De este modo, buscamos las etiquetas que nos son de utilidad para extraer el contenido deseado, en mi caso la etiqueta div de clase sc-1uhtbxc-0.hpNmeK y completamos con el siguiente codigo:





In [ ]:
def parse(self, response): #parse es la funcion que el scrapper va a ejecutar para realizar el ejercicio de scrapping.
    div_selector = response.css('div.sc-1uhtbxc-0.hpNmeK') #implica guardar la respuesta del contenido de la etiqueta de css mencionada en una variable
    extracted_text = div_selector.css('::text').get() #extrae unicamente la parte de texto y elimina el resto del contenido
    print("Texto extraído:", extracted_text) #imprime el texto, para verificar que estamos extrayendo lo deseado

A partir de aqui, debemos correr el codigo posicionandonos dentro de la carpeta zonaprop con la terminal (bash o el shell, es lo mismo) y ejecutando el siguiente codigo "scrapy crawl zonaprop_ret". <br>
El resultado es un codigo extenso que indica entre otras cosas lo siguiente:
<ul>
<li>2023-11-15 17:44:29 [scrapy.core.engine] INFO: Spider opened ------ Aca se abre scrapy. </li>
<li>2023-11-15 17:44:29 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min) </li>
<li>2023-11-15 17:44:29 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023 </li>
<li>2023-11-15 17:44:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.zonaprop.com.ar/robots.txt> (referer: None) ------Busca el archivo robots.txt ya que las paginas suelen subir ese archivo donde describe las limitaciones de acceso, y no lo encuentra </li>
<li>2023-11-15 17:44:31 [scrapy.core.engine] DEBUG: Crawled (403) <GET https://www.zonaprop.com.ar/ph-alquiler-capital-federal.html> (referer: None) ------ obtiene el error 403, implicando que pudo acceder pero algo no anda bien </li>
<li>2023-11-15 17:44:32 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <403 https://www.zonaprop.com.ar/ph-alquiler-capital-federal.html>: HTTP status code is not handled or not allowed ----- recibe la prohibicion de scrapeo de la pagina </li>
<li>2023-11-15 17:44:32 [scrapy.core.engine] INFO: Closing spider (finished) </li> </ul>

En este momento es hora de replantear el objetivo, ya que no vamos a poder descargar informacion de zonaprop, y como ya me paso varias veces este error al intentar descargar desde otras paginas asi como al intentar conectar una API de Twitter, vamos a proceder a cambiar el trabajo.

### Redireccionamiento

El nuevo objetivo es el siguiente: Descargar y analizar textos de wikipedia para entrenar a nuestra ML para que pueda distinguir que textos refieren a musicos, cuales a actores, cuales a pintores y cuales a escritores.

Para ello vamos a necesitar muchos links de wikipedia, y como nos gusta mas la cabeza que los pies, se los pediremos de la siguiente forma a chatGPT:

"Sos un machine learning developer, y estas haciendo un data set para una clasificación de texto. Necesitas armar el conjunto de datos. Necesito que me des 50 links de wikipedia en ingles, relacionados con el tema Musicos. Me los tenes que dar en formato csv con dos columnas, la primera con las URL y la segunda con la Clase, que en este caso sería Musician."

Con esto te va a dar los 50 links (si no lo hace indica que es a modo ejemplo y que necesitas los links explicitos). Y para evitar crear el csv, aunque el entregable fue en formato csv, vamos a usar python... primero repetimos lo mismo para Actor, Writer y Painter.

In [10]:
import csv 
import pandas as pd
# se puede pedir a chat gpt que te vuelva a enviar el listado de url explicitas entre comillas y con coma al final.
musicians = [
    'https://en.wikipedia.org/wiki/Elvis_Presley',
    'https://en.wikipedia.org/wiki/The_Beatles',
    'https://en.wikipedia.org/wiki/Bob_Dylan',
    'https://en.wikipedia.org/wiki/Michael_Jackson',
    'https://en.wikipedia.org/wiki/Prince_(musician)',
    'https://en.wikipedia.org/wiki/Madonna_(entertainer)',
    'https://en.wikipedia.org/wiki/Johnny_Cash',
    'https://en.wikipedia.org/wiki/Queen_(band)',
    'https://en.wikipedia.org/wiki/David_Bowie',
    'https://en.wikipedia.org/wiki/Beyonc%C3%A9',
    'https://en.wikipedia.org/wiki/Led_Zeppelin',
    'https://en.wikipedia.org/wiki/Jimi_Hendrix',
    'https://en.wikipedia.org/wiki/The_Rolling_Stones',
    'https://en.wikipedia.org/wiki/Whitney_Houston',
    'https://en.wikipedia.org/wiki/Freddie_Mercury',
    'https://en.wikipedia.org/wiki/John_Lennon',
    'https://en.wikipedia.org/wiki/Pink_Floyd',
    'https://en.wikipedia.org/wiki/George_Michael',
    'https://en.wikipedia.org/wiki/Prince_Royce',
    'https://en.wikipedia.org/wiki/Aretha_Franklin',
    'https://en.wikipedia.org/wiki/Bruce_Springsteen',
    'https://en.wikipedia.org/wiki/U2',
    'https://en.wikipedia.org/wiki/Rihanna',
    'https://en.wikipedia.org/wiki/Bruno_Mars',
    'https://en.wikipedia.org/wiki/Adele',
    'https://en.wikipedia.org/wiki/Taylor_Swift',
    'https://en.wikipedia.org/wiki/Kanye_West',
    'https://en.wikipedia.org/wiki/Lady_Gaga',
    'https://en.wikipedia.org/wiki/Ed_Sheeran',
    'https://en.wikipedia.org/wiki/Drake_(musician)',
    'https://en.wikipedia.org/wiki/Justin_Bieber',
    'https://en.wikipedia.org/wiki/Coldplay',
    'https://en.wikipedia.org/wiki/Nirvana_(band)',
    'https://en.wikipedia.org/wiki/Eminem',
    'https://en.wikipedia.org/wiki/Jay-Z',
    'https://en.wikipedia.org/wiki/Elton_John',
    'https://en.wikipedia.org/wiki/AC/DC',
    'https://en.wikipedia.org/wiki/Linkin_Park',
    'https://en.wikipedia.org/wiki/Metallica',
    'https://en.wikipedia.org/wiki/Maroon_5',
    'https://en.wikipedia.org/wiki/Christina_Aguilera',
    'https://en.wikipedia.org/wiki/Selena_Gomez',
    'https://en.wikipedia.org/wiki/Shakira',
    'https://en.wikipedia.org/wiki/Ricky_Martin',
    'https://en.wikipedia.org/wiki/Pitbull_(rapper)',
    'https://en.wikipedia.org/wiki/Jennifer_Lopez',
    'https://en.wikipedia.org/wiki/Frank_Sinatra',
    'https://en.wikipedia.org/wiki/Frank_Zappa',
    'https://en.wikipedia.org/wiki/Billie_Holiday',
    'https://en.wikipedia.org/wiki/Kanye_West',
]

painters = [
    'https://en.wikipedia.org/wiki/Leonardo_da_Vinci',
    'https://en.wikipedia.org/wiki/Pablo_Picasso',
    'https://en.wikipedia.org/wiki/Vincent_van_Gogh',
    'https://en.wikipedia.org/wiki/Claude_Monet',
    'https://en.wikipedia.org/wiki/Rembrandt',
    'https://en.wikipedia.org/wiki/Michelangelo',
    'https://en.wikipedia.org/wiki/Gustav_Klimt',
    'https://en.wikipedia.org/wiki/Salvador_Dal%C3%AD',
    'https://en.wikipedia.org/wiki/Frida_Kahlo',
    'https://en.wikipedia.org/wiki/Jackson_Pollock',
    'https://en.wikipedia.org/wiki/Georgia_O%27Keeffe',
    'https://en.wikipedia.org/wiki/Pierre-Auguste_Renoir',
    'https://en.wikipedia.org/wiki/Edvard_Munch',
    'https://en.wikipedia.org/wiki/Henri_Matisse',
    'https://en.wikipedia.org/wiki/Camille_Pissarro',
    'https://en.wikipedia.org/wiki/Wassily_Kandinsky',
    'https://en.wikipedia.org/wiki/Andy_Warhol',
    'https://en.wikipedia.org/wiki/Caravaggio',
    'https://en.wikipedia.org/wiki/Goya',
    'https://en.wikipedia.org/wiki/Edgar_Degas',
    'https://en.wikipedia.org/wiki/Titian',
    'https://en.wikipedia.org/wiki/Sandro_Botticelli',
    'https://en.wikipedia.org/wiki/Joan_Mir%C3%B3',
    'https://en.wikipedia.org/wiki/Peter_Paul_Rubens',
    'https://en.wikipedia.org/wiki/Paul_Gauguin',
    'https://en.wikipedia.org/wiki/Kazimir_Malevich',
    'https://en.wikipedia.org/wiki/Grant_Wood',
    'https://en.wikipedia.org/wiki/Joaqu%C3%ADn_Sorolla',
    'https://en.wikipedia.org/wiki/C%C3%A9zanne',
    'https://en.wikipedia.org/wiki/Hieronymus_Bosch',
    'https://en.wikipedia.org/wiki/Jan_van_Eyck',
    'https://en.wikipedia.org/wiki/Raphael',
    'https://en.wikipedia.org/wiki/%C3%89douard_Manet',
    'https://en.wikipedia.org/wiki/Piet_Mondrian',
    'https://en.wikipedia.org/wiki/Georges_Seurat',
    'https://en.wikipedia.org/wiki/Amedeo_Modigliani',
    'https://en.wikipedia.org/wiki/Diego_Vel%C3%A1zquez',
    'https://en.wikipedia.org/wiki/Albrecht_D%C3%BCrer',
    'https://en.wikipedia.org/wiki/Francisco_Goya',
    'https://en.wikipedia.org/wiki/Edgar_Degas',
    'https://en.wikipedia.org/wiki/Titian',
    'https://en.wikipedia.org/wiki/Sandro_Botticelli',
    'https://en.wikipedia.org/wiki/Joan_Mir%C3%B3',
    'https://en.wikipedia.org/wiki/Peter_Paul_Rubens',
    'https://en.wikipedia.org/wiki/Paul_Gauguin',
    'https://en.wikipedia.org/wiki/Kazimir_Malevich',
    'https://en.wikipedia.org/wiki/Grant_Wood',
    'https://en.wikipedia.org/wiki/Joaqu%C3%ADn_Sorolla',
    'https://en.wikipedia.org/wiki/C%C3%A9zanne',
    'https://en.wikipedia.org/wiki/Hieronymus_Bosch'
    ]

actors = [
    "https://en.wikipedia.org/wiki/Robert_De_Niro",
    "https://en.wikipedia.org/wiki/Meryl_Streep",
    "https://en.wikipedia.org/wiki/Leonardo_DiCaprio",
    "https://en.wikipedia.org/wiki/Cate_Blanchett",
    "https://en.wikipedia.org/wiki/Tom_Hanks",
    "https://en.wikipedia.org/wiki/Julia_Roberts",
    "https://en.wikipedia.org/wiki/Denzel_Washington",
    "https://en.wikipedia.org/wiki/Natalie_Portman",
    "https://en.wikipedia.org/wiki/Johnny_Depp",
    "https://en.wikipedia.org/wiki/Morgan_Freeman",
    "https://en.wikipedia.org/wiki/Emma_Stone",
    "https://en.wikipedia.org/wiki/Brad_Pitt",
    "https://en.wikipedia.org/wiki/Cate_Blanchett",
    "https://en.wikipedia.org/wiki/Tom_Cruise",
    "https://en.wikipedia.org/wiki/Jennifer_Lawrence",
    "https://en.wikipedia.org/wiki/Will_Smith",
    "https://en.wikipedia.org/wiki/Angelina_Jolie",
    "https://en.wikipedia.org/wiki/Sean_Connery",
    "https://en.wikipedia.org/wiki/Kate_Winslet",
    "https://en.wikipedia.org/wiki/George_Clooney",
    "https://en.wikipedia.org/wiki/Charlize_Theron",
    "https://en.wikipedia.org/wiki/Harrison_Ford",
    "https://en.wikipedia.org/wiki/Anne_Hathaway",
    "https://en.wikipedia.org/wiki/Matt_Damon",
    "https://en.wikipedia.org/wiki/Nicole_Kidman",
    "https://en.wikipedia.org/wiki/Anthony_Hopkins",
    "https://en.wikipedia.org/wiki/Sandra_Bullock",
    "https://en.wikipedia.org/wiki/Mark_Wahlberg",
    "https://en.wikipedia.org/wiki/Scarlett_Johansson",
    "https://en.wikipedia.org/wiki/Chris_Hemsworth",
    "https://en.wikipedia.org/wiki/Julianne_Moore",
    "https://en.wikipedia.org/wiki/Dwayne_Johnson",
    "https://en.wikipedia.org/wiki/Reese_Witherspoon",
    "https://en.wikipedia.org/wiki/Christian_Bale",
    "https://en.wikipedia.org/wiki/Emily_Blunt",
    "https://en.wikipedia.org/wiki/Chris_Evans",
    "https://en.wikipedia.org/wiki/Emma_Watson",
    "https://en.wikipedia.org/wiki/Russell_Crowe",
    "https://en.wikipedia.org/wiki/Amy_Adams",
    "https://en.wikipedia.org/wiki/Matthew_McConaughey",
    "https://en.wikipedia.org/wiki/Jessica_Chastain",
    "https://en.wikipedia.org/wiki/Ryan_Reynolds",
    "https://en.wikipedia.org/wiki/Naomi_Watts",
    "https://en.wikipedia.org/wiki/Chadwick_Boseman",
    "https://en.wikipedia.org/wiki/Kristen_Stewart",
    "https://en.wikipedia.org/wiki/Idris_Elba",
    "https://en.wikipedia.org/wiki/Margot_Robbie",
    "https://en.wikipedia.org/wiki/Chris_Pratt",
    "https://en.wikipedia.org/wiki/Keanu_Reeves",
    "https://en.wikipedia.org/wiki/Gal_Gadot"
    ]

writers = [
    "https://en.wikipedia.org/wiki/William_Shakespeare",
    "https://en.wikipedia.org/wiki/Jane_Austen",
    "https://en.wikipedia.org/wiki/Charles_Dickens",
    "https://en.wikipedia.org/wiki/Fyodor_Dostoevsky",
    "https://en.wikipedia.org/wiki/Leo_Tolstoy",
    "https://en.wikipedia.org/wiki/Mark_Twain",
    "https://en.wikipedia.org/wiki/Charlotte_Bront%C3%AB",
    "https://en.wikipedia.org/wiki/George_Orwell",
    "https://en.wikipedia.org/wiki/J.K._Rowling",
    "https://en.wikipedia.org/wiki/Homer",
    "https://en.wikipedia.org/wiki/J.R.R._Tolkien",
    "https://en.wikipedia.org/wiki/Agatha_Christie",
    "https://en.wikipedia.org/wiki/Ernest_Hemingway",
    "https://en.wikipedia.org/wiki/John_Steinbeck",
    "https://en.wikipedia.org/wiki/Virginia_Woolf",
    "https://en.wikipedia.org/wiki/Gabriel_Garc%C3%ADa_M%C3%A1rquez",
    "https://en.wikipedia.org/wiki/George_R.R._Martin",
    "https://en.wikipedia.org/wiki/Jane_Eyre",
    "https://en.wikipedia.org/wiki/Edgar_Allan_Poe",
    "https://en.wikipedia.org/wiki/Arthur_Conan_Doyle",
    "https://en.wikipedia.org/wiki/Franz_Kafka",
    "https://en.wikipedia.org/wiki/James_Joyce",
    "https://en.wikipedia.org/wiki/Emily_Bront%C3%AB",
    "https://en.wikipedia.org/wiki/Ray_Bradbury",
    "https://en.wikipedia.org/wiki/Isaac_Asimov",
    "https://en.wikipedia.org/wiki/Charles_Bukowski",
    "https://en.wikipedia.org/wiki/William_Faulkner",
    "https://en.wikipedia.org/wiki/F.%C2%A0Scott_Fitzgerald",
    "https://en.wikipedia.org/wiki/T.S._Eliot",
    "https://en.wikipedia.org/wiki/Graham_Greene",
    "https://en.wikipedia.org/wiki/Herman_Melville",
    "https://en.wikipedia.org/wiki/Albert_Camus",
    "https://en.wikipedia.org/wiki/John_Milton",
    "https://en.wikipedia.org/wiki/Walt_Whitman",
    "https://en.wikipedia.org/wiki/Kurt_Vonnegut",
    "https://en.wikipedia.org/wiki/George_Bernard_Shaw",
    "https://en.wikipedia.org/wiki/Jules_Verne",
    "https://en.wikipedia.org/wiki/Aldous_Huxley",
    "https://en.wikipedia.org/wiki/Maya_Angelou",
    "https://en.wikipedia.org/wiki/Giovanni_Boccaccio",
    "https://en.wikipedia.org/wiki/Anne_Frank",
    "https://en.wikipedia.org/wiki/Aristotle",
    "https://en.wikipedia.org/wiki/John_Keats",
    "https://en.wikipedia.org/wiki/Thomas_Hardy",
    "https://en.wikipedia.org/wiki/H.P._Lovecraft",
    "https://en.wikipedia.org/wiki/Margaret_Atwood",
    "https://en.wikipedia.org/wiki/G%C3%BCnter_Grass",
    "https://en.wikipedia.org/wiki/Rabindranath_Tagore",
    "https://en.wikipedia.org/wiki/Haruki_Murakami",
    "https://en.wikipedia.org/wiki/Albert_Camus"
    ]

#como pasar esto a csv?

categorias = [[musicians,'Musician'],[painters,'Painter'],[actors,'Actor'],[writers,'Writer']]
a_csv = []

for i in categorias:
    for j in i[0]:
        a_csv.append([j,i[1]])

df = pd.DataFrame(a_csv,columns=['URL',"Type"])

df.to_csv('Datasets/conjunto_enlaces.csv',index=False) #previamente hay que crear la carpeta Datasets


### Web Scrapping

Vamos a intentar hacerlo con las dos herramientas que conocemos, que sirven para lo mismo.

#### Scrapy

Abrimos en terminal como hicimos anteriormente y nos vamos a la carpeta del proyecto para ejecutar el siguiente codigo:

In [ ]:
scrapy startproject wikiscrap
cd wikiscrap
scrapy genspider wikispider wikipedia.com
#esto generara el scrapy y spider correspondientes que debemos editar.

# ------------ Edicion del spider esta explicado en cada archivo -----------------------------------
# los archivos modificados a revisar por el usuario (en orden) son: items.py, pipelines.py, setting.py y wikispider.py
# una vez modificados los archivos, avanzamos con el siguiente codigo
scrapy crawl wikispider

In [43]:
df1 = pd.read_csv('wikiscrap/output.csv')

df1.head(15) # para verificar que todo se haya exportado correctamente

,url,entity_class,paragraph
0,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,"(January 8, 1935 – August 16, 1977), commonly..."
1,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,", was an American singer and actor. Known as t..."
2,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,""", he is regarded as"
3,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,. Presley's energized interpretations of songs...
4,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,", brought both great success and"
5,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,.
6,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,Presley was born in
7,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,; his family relocated to
8,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,", when he was 13. His music career began there..."
9,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,with producer


Como veran, todo salio perfecto!!! VAmos por el proximo metodo, mas simple.

#### Request + bs4

Al ser todo en python, crearemos funciones que pueden guardar como modulo posteriormente a su conveniencia.

In [46]:
#siempre para que no se olviden, vamos a importar las librerias necesarias (aunque sea redundante)

import pandas as pd
import requests
import bs4 as BeautifulSoup4

def obtener_sentencias(ruta_csv):

    df2 = pd.read_csv(ruta_csv)
    registros = []

    for _, row in df2.iterrows():
        url = row["URL"]
        tipo = row["Type"]

        req = requests.get(url)
        if req.status_code == 200:
            html = BeautifulSoup4.BeautifulSoup(req.text,'html.parser')
            parrafos = [para.text for para in html.find_all('p')]
        
            for parrafo in parrafos:
                sentencias = parrafo.split('\n')

                for sentencia in sentencias:
                    if sentencia == '':
                        continue

                    sentencia = sentencia.strip()
                    registros.append([url,tipo,sentencia])
    
    df_reg = pd.DataFrame(registros)
    return df_reg

obtener_sentencias('Datasets/conjunto_enlaces.csv')   

,0,1,2
0,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,"Elvis Aaron Presley[a] (January 8, 1935 – Augu..."
1,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,"Presley was born in Tupelo, Mississippi; his f..."
2,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,"In November 1956, Presley made his film debut ..."
3,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,Having sold roughly 500 million records worldw...
4,https://en.wikipedia.org/wiki/Elvis_Presley,Musician,"Elvis Aaron Presley was born on January 8, 193..."
...,...,...,...
15895,https://en.wikipedia.org/wiki/Albert_Camus,Writer,Camus's novels and philosophical essays are st...
15896,https://en.wikipedia.org/wiki/Albert_Camus,Writer,Although Camus has been linked to anti-Soviet ...
15897,https://en.wikipedia.org/wiki/Albert_Camus,Writer,American heavy metal band Avenged Sevenfold ha...
15898,https://en.wikipedia.org/wiki/Albert_Camus,Writer,Albert Camus also served as the inspiration fo...
